In [1]:
# CIS 545

# CIS 545 Homework 3: Advanced Portion

## Step 5: tf-idf search engine
Now that you have implemented PageRank, which tells you how authoritative websites are, you are going to build the other very important part of a search engine. Namely, this part will retrieve the documents that are _relevant_ to a given query. Simply put, this part will give you a subset of webpages that are presumed to be relevant to the query and then PageRank can rerank them according to how much you value authoritativeness over relevance. The methods in this section are a basis of both _keyword search_, and also _document clustering_.  (We’ll see a lot more about clustering in a few weeks.)

## Step 5.1: Preliminaries

### Step 5.1.1 Load nltk and some helpful tools

The cell below gives you some tools and assigns some variables that you will need later. You do not need to modify this cell provided that you have everything installed already.

In [1]:
#import csv
import pandas as pd
import numpy as np
import nltk
#from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [2]:
# [hw3-stopfilesetup]
# This imports your data and sets up nltk parsers
# You should not have to modify the cell

"""
# Returns True if the input (string) parameter has
# any sort of letter in it, else returns False.
"""
def has_letter(x):
    return re.match('.*[a-zA-Z].*',x) != None

# Stopwords are words we will ignore for search
# purposes, because they are too common to be useful
stopwords = set()

# Import the data - the S3 link is only for grading, and you should still use the stopwords.txt for now
try:
    stop_file = open('stopwords.txt')
except:
    stop_file = pd.read_csv('https://s3.amazonaws.com/cis545-hw3data/stopwords.txt', header = None)[0].tolist()

for line in stop_file:
    stopwords.add(line.strip())

# The NLTK parser breaks apostrophe-s into a separate "word"
# so we'll want to add it to the list... Though it's technically
# not a stop word in the traditional sense.
stopwords.add("'s")

# Create the word stemmer
stemmer = PorterStemmer()

In [3]:
# print(stopwords)

### Step 5.1.2. Import text files, clean them, and store in a dictionary 

As a "corpus" we fetched some data from Wikipedia, based on currently
trendy topics.  Each topic had multiple interpretations, some of which 
we suspected would "intersect" in interesting ways (e.g., Trump/Putin, Cloud/Google, 
Cloud/Climate).  Others had various interpretations (e.g., there are many types of 
Football).  See _Wikipedia.ipynb_ for the original download code. - these are from **text.zip**

Selected topics (for which the top-10 matches were returned by Wikipedia) were:

 * Pennsylvania
 * Trump
 * Apple
 * Google
 * Farm
 * Climate
 * Cloud
 * Football
 * Government
 * Putin
 
Please write the function `clean_article(article)` that takes a string as input and then:

1. tokenizes it using `nltk.word_tokenize` (this one is not for Twitter)
2. converts it to lower case
3. removes stopwords that are present in the set from the cell above
4. uses `stemmer` (defined above) to cut the word down to its stem
5. uses `has_letter` (defined above) to remove words that don't have any letters
6. keeps only those words with length greater than 1 and less than 20.

It is important that you perform these steps in order so your result will be the same as ours!

In [4]:
# [hw3-clean_article]
# TODO: Write the clean_article function, noting that 'article' is a string input
# for i in range(0,len(clean_incidents)):
#     word_raw = nltk.word_tokenize(clean_incidents[i])#nltk 停用词问题
#     word = [w for w in word_raw if(w not in stopwords.words('english'))]

import os
import requests
import re

#words = stopwords.words('english')

def clean_article(article):
    tokens = nltk.word_tokenize(article)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if(token not in stopwords)]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = [token for token in tokens if(has_letter(token)==True)]
    tokens = [token for token in tokens if(len(token)>1 and len(token)<20)]
    return tokens
    #print(tokens)

In [5]:
# [hw3-loadfiles]
# Run: Load files for test cases 
# You should unzip test.zip prior to running this

docs = {}

try:
    # Local directory - to be used during homework
    for filename in os.listdir('text'):
        # Opens files and cleans to import
        file = open('text/' + filename)
        docs[filename] = clean_article(file.read())
    
except FileNotFoundError:
    # Used for grading only - the link will not be active until you finish submitting!
    # Pulls list of files from AWS
    r = requests.get('https://s3.amazonaws.com/cis545-hw3data/list_of_texts.txt')
    r.encoding = 'utf-8'
    filenames = r.text.split('\n')
    for textfile in filenames:
        
        # Determines filenames
        filename = textfile.strip()
        filepath = filename.replace('+', '%2B')
        filepath = filepath.replace(',', '%2C')
        filepath = filepath.replace(' ', '+')
        
        # Extracts our data
        filepath = 'https://s3.amazonaws.com/cis545-hw3data/text/' + filepath
        r = requests.get(filepath)
        r.encoding = 'utf-8'        
        
        # Cleans UTF-8 data
        docs[filename] = clean_article(r.text)

print ("All files loaded")

All files loaded


In [7]:
# [hw3-pad]
# This cell is for grading purposes only! Please DO NOT modify it.


print("[CIS 545 Homework 3] Padding")

[CIS 545 Homework 3] Padding


In [8]:
# [test-docs]
# [CIS 545 Homework 3] Test Case
# Testing your values in docs - that things were loaded correctly

assert(docs['Apple.txt'][20] == 'central')


In [9]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

[CIS 545 Homework 3] Test Case - 2 points


NameError: name 'score' is not defined

## Step 5.2. Generate the Vocabulary

As discussed in class, natural language words follow a **Zipfian distribution**, which means that many, sometimes over half, of the unqiue words in a collection of documents only occur once. Limiting the **vocabulary**, or the list of words that are considered for a data science task, often improves both the efficiency and accuracy of the system. Now, we are going to limit the vocabulary of our search engine using a minimum number of occurrences. In the next cell, you should:

1. Create a single list containing the words of each (cleaned) document in succession.
2. Count occurrences of each word. You may use `nltk.FreqDist` if you wish but you do not have to.
3. Remove all words that only occurred once. You may use `hapaxes` if you wish but you do not have to.
4. Make a list called `wordids` that contains the words in decreasing frequency order. For example, the most frequent word in this document collection is "appl", so `wordids[0] = 'appl'`.
5. Make a dictionary called `lexicon` that has words as keys and their indices in `wordids` as values.

In [6]:
# [hw3-create-lexicon]
# TODO: Create your vocabulary here. 
# We will test the list named wordids and the dictionary named lexicon
from collections import Counter
word_list = []
for key, value in docs.items():
    #print(value)
    word_list.extend(value)
#print(word_list)
word_freq = nltk.FreqDist(word_list).most_common()
#print(word_freq)
wordids = [key for key,value in word_freq if(value !=1)]
#print(wordids)
lexicon = {}
i=0
for item in wordids:
    lexicon[item]=i
    i=i+1
#print(lexicon)

In [7]:
# [test-lexicon1]
# [CIS 545 Homework 3] Test Case (test case pad below!)
# Testing your wordids (public test only)

print("There are", len(lexicon), "words in your lexicon.")
display(wordids[:10])
assert(wordids[3] == 'appl')
assert(wordids[7] == 'footbal')

There are 11074 words in your lexicon.


['googl',
 'trump',
 'state',
 'appl',
 'cloud',
 'use',
 'also',
 'footbal',
 'first',
 'new']

In [12]:
# [CIS 545 Test Cases] (1 pts)
print("[CIS 545 Homework 3] Test Case - 1 points")
score = score + 1

[CIS 545 Homework 3] Test Case - 1 points


NameError: name 'score' is not defined

In [13]:
# [test-lexicon2]
# [CIS 545 Homework 3] Test Case (test case pad below!)
# Testing your wordids (hidden tests)


In [14]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

[CIS 545 Homework 3] Test Case - 2 points


NameError: name 'score' is not defined

## Step 5.3. Term Frequencies

We are now going to create a document-term matrix. Write a function `doc_vector(content, lexicon)` that given a cleaned article (as a list of words) and the lexicon, creates a vector of term frequencies. We will define **term frequency** as the number of times the word occurs in the document divided by the number of times the most frequent word in the document occurs. For example, in the document `['a', 'b', 'b']`, 'a' has a term frequency of 0.5 and 'b' has a term frequency of 1.0.

For greatest portability, please use the length of the vocabulary (`lexicon`) to determine the length of the output vector.

In [11]:
def doc_vector(content, lexicon):
    term_frequency = [0]*len(lexicon)
    #print(term_frequency)
    word_list = nltk.FreqDist(content).most_common()
    largest = word_list[0][1]
    for key,value in word_list:
        if key in lexicon:
            term_frequency[lexicon[key]]=value/largest
#     print(term_frequency)
    return term_frequency

The next cell is going to assemble your document-term matrix. It is also going to test whether your `doc_vector` function can handle the case in which you provide an "article" with no words in the vocabulary. This test is visible to you, so if you pass it, you are fine. You should not need to edit it.

The hidden test is a more standard use case in which the article contains some valid word or words.

In [12]:
# [hw3-assemblevector]
# Assembles document-term matrix

vectors = []
doclist = []

for topic in docs:
    doclist.append(topic)
    vectors.append(doc_vector(docs[topic], lexicon))

vectors = np.array(vectors)

In [13]:
print(len(lexicon))

11074


In [ ]:
print(doclist)

In [15]:
clean_article('to be or not to Be')

[]

In [14]:
# [test-hamlet]
# Tests against an example with some valid words

assert(sum(doc_vector(clean_article('to be or not to Be'), lexicon)) == 0)

IndexError: list index out of range

In [ ]:
# [CIS 545 Test Cases] (1 pt)
print("[CIS 545 Homework 3] Test Case - 1 point")
score = score + 1

In [ ]:
# [test-vector1]
# Tests against a text example


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

The next cell is going to display the counts of all words in the Putinism document.

In [16]:
# [hw3-createlistwords]
# Creates the counts of words
# This should be the Putinism document

putin_docid = doclist.index("Putinism.txt")
list_of_words = []
for ident in range(0, len(lexicon)):
    if vectors[putin_docid, ident] > 0:
        list_of_words.append(wordids[ident] + ' x ' + str(vectors[putin_docid, ident]))
        
display(list_of_words[:15])

['state x 0.7916666666666666',
 'use x 0.08333333333333333',
 'also x 0.25',
 'first x 0.041666666666666664',
 'new x 0.16666666666666666',
 'includ x 0.041666666666666664',
 'can x 0.08333333333333333',
 'putin x 1.0',
 'one x 0.16666666666666666',
 'year x 0.041666666666666664',
 'govern x 0.20833333333333334',
 'unit x 0.08333333333333333',
 'compani x 0.041666666666666664',
 'servic x 0.20833333333333334',
 'may x 0.16666666666666666']

In [17]:
# [test-listofwords1]
# Test cases for values in vectors and list of words

### BEGI1 HIDDEN TESTS
ep = 0.02 # Error amount
assert(abs(vectors[2][2]*14 - 3) < ep) # Should be 0
assert(abs(vectors[12][16] * 6000 - 99.44) < ep/2) # Should be ~0.0075

In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

In [ ]:
# [test-listofwords2]
# Test cases for values in vectors and list of words


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

## Step 5.4. Inverse Document Frequencies

We would like to give **rare words** a higher weight than ones found in all documents.  To measure a word’s rareness, we will develop a metric called **inverse document frequency (idf)**.  In its simplest form, a word’s idf is a ratio between the total number of documents, and how many documents include a word.  (Note that the idf is independent of a given document -- it is a measure of the word’s popularity across the full set of documents, sometimes called a **corpus**.)  Typically, we don’t directly use the ratio, however -- instead we use its base-10 logarithm.  Thus, we define:

$$idf(w) = \log(\frac{\text{total number of docs}}{\text{number of docs containing }w})$$

Now compute a single vector `idf` representing, for each word, its idf within the corpus.

In [18]:
# [hw3-idf]
# TODO: Compute inverse document frequencies
import math

def func_idf(total_num, x):
    count=0
    for key, value in docs.items():
        if x in value:
            count=count+1
    idf = math.log10(total_num/count)
    return idf


total_num = len(docs)
#print(total_num)
idf = [func_idf(total_num, w) for w in wordids]


In [19]:
# [test-idf1]
# Test cases for values in IDF

# Print first 20 to view 
for i in range(0, 20):
    print(wordids[i], idf[i])


googl 0.7269987279362623
trump 0.7518223116612945
state 0.06319314066349449
appl 0.5843312243675307
cloud 0.5843312243675307
use 0.06845738065585169
also 0.023229840718474823
footbal 0.5509074688805811
first 0.06845738065585169
new 0.05799194697768673
climat 0.5351132016973491
includ 0.05285230732527567
can 0.11303951330859223
citi 0.20411998265592482
putin 0.8683278807327317
one 0.07378621416091864
game 0.38021124171160603
year 0.07378621416091864
govern 0.24987747321659987
unit 0.10145764075877703


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

In [ ]:
# [test-idf2]
# Test cases for values in IDF


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

## Step 5.5. Produce Ranked Results

As we described previously, standard search simply takes a keyword query and treats it as another document.  This how you would generate a query vector:

To compute the similarity between two documents, we use the cosine of the angle between their vectors, given by:

$$sim(d_1,d_2) = \frac{d_1 \cdot d_2}{||d_1||\;||d_2||}$$

where $||d_i||$ is the Frobenius ($L_2$) norm of the document vector. Write a function `search(vectors, doclist, idf, query, num_results)` that, when given a 2D array of document vectors, a list of document names, idf scores for each word, a query vector, and the desired number of results:

1. Creates a Pandas DataFrame with schema (docid, docname, score) for the results of matching the query against each document.
2. Scales the words in the document and query vectors by their idfs.
3. Computes the cosine similarity score for the query against each document. Hint: Uses Numpy multiplication (`*` between arrays), dot product (`@` or `np.dotproduct()`) and other operations (`+`, `np.linalg.norm()` for vector norms, etc.)
4. Adds each document ID and score to the DataFrame.
5. Sorts the DataFrame by descending score.
6. Returns the first `num_results` results.

In [63]:
# [hw3-search]
# Uses the cosine similarity to create the num_results

import pandas as pd

def search(vectors, doclist, idf, query, num_results):
    result=pd.DataFrame({'docid':range(0,len(doclist)),'docname':doclist,'score':[0.0]*len(doclist)})
#     print(result)
#     print(type(vectors), type(doclist), type(idf), type(query))
    idf_arr=np.array(idf)
    query_idf=np.array(query)*idf_arr
#     vectors_idf=[]
    i=0
    for doc_vec in vectors:
        vector_idf=np.array(doc_vec)*idf_arr
        sim=np.dot(vector_idf,query_idf.transpose())/(np.linalg.norm(vector_idf)*np.linalg.norm(query_idf))
#         print(sim)
        result.iloc[i,2]=sim
#         print(result.iloc[i]['score'].round(5),i)
        i=i+1
    result=result.sort_values(by='score',ascending=False)[:num_results]
#     print(type(result['score']),type(result['score'][0]))
    return result
    

In [64]:
# [test-search1]
# Create a search query and test the values

result = search(vectors, doclist, idf, doc_vector(clean_article('apple computer steve jobs'), lexicon), 10)
display(result)


,docid,docname,score
8,8,Apple Inc..txt,0.472841
4,4,Apple II series.txt,0.357818
75,75,Apple.txt,0.288584
81,81,Macintosh.txt,0.264923
31,31,In the Cloud.txt,0.213965
38,38,Cloud computing.txt,0.213965
22,22,Fiona Apple.txt,0.213672
61,61,Apples to Apples.txt,0.213044
87,87,IPhone.txt,0.111595
58,58,IPad.txt,0.100798


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

In [65]:
# [test-search2]
# Create a search query and test the values

result = search(vectors, doclist, idf, doc_vector(clean_article('Trump Putin'), lexicon), 10)
display(result)


,docid,docname,score
76,76,Vladimir Putin.txt,0.662309
23,23,Public image of Vladimir Putin.txt,0.658737
72,72,Trump.txt,0.610337
24,24,Donald Trump.txt,0.610337
0,0,Political career of Vladimir Putin.txt,0.593516
40,40,The Trump Organization.txt,0.591288
37,37,Russia under Vladimir Putin.txt,0.567728
62,62,Family of Donald Trump.txt,0.533129
74,74,Ivanka Trump.txt,0.516982
32,32,The Putin Interviews.txt,0.494010


In [ ]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2